In [1]:
import pandas as pd
import numpy as np
import os
import random
from copy import deepcopy
import math
from itertools import product
import pickle

import joblib
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from supervised.automl import AutoML

# 데이터 분리

In [2]:
df=pd.read_csv('./valiData/merged_5_0.csv',index_col=0)

In [3]:
df.drop(['application_id', 'user_id','insert_time',
               'loanapply_insert_time', 'bank_id', 'product_id',
               ], axis=1, inplace=True)

In [4]:
model1_train_X = df[(df['is_applied'].isnull()==False)&((df['credit_score'].isnull()==False))].loc[:, df.columns !='is_applied']
model1_train_y = df[(df['is_applied'].isnull()==False)&((df['credit_score'].isnull()==False))]['is_applied']
model2_train_X = df[(df['is_applied'].isnull()==False)&((df['credit_score'].isnull()==True))].loc[:, df.columns !='is_applied']
model2_train_X.drop(['credit_score'], axis=1,inplace=True)
model2_train_y = df[(df['is_applied'].isnull()==False)&((df['credit_score'].isnull()==True))]['is_applied']

model1_target= df[(df['is_applied'].isnull()==True)&((df['credit_score'].isnull()==False))].loc[:, df.columns !='is_applied']
model2_target= df[(df['is_applied'].isnull()==True)&((df['credit_score'].isnull()==True))].loc[:, df.columns !='is_applied']
model2_target.drop(['credit_score'], axis=1,inplace=True)

In [5]:
model1_train_X = model1_train_X.sample(20_0000, random_state=42)
model1_train_y = model1_train_y.sample(20_0000, random_state=42)
model2_train_X = model2_train_X.sample(10_0000, random_state=42)
model2_train_y = model2_train_y.sample(10_0000, random_state=42)

In [6]:
model1_train_X.shape, model1_train_y.shape

((200000, 30), (200000,))

In [7]:
model2_train_X.shape, model2_train_y.shape

((100000, 29), (100000,))

In [8]:
model1_target.shape, model2_target.shape, 

((3000756, 30), (256483, 29))

In [9]:
model1 = AutoML(mode='Compete')
model1.fit(model1_train_X, model1_train_y)

Linear algorithm was disabled.
AutoML directory: AutoML_7
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree logloss 0.189277 trained in 3.04 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree logloss 0.191747 trained in 27.57 seconds
2_DecisionTree logloss 0.186139 trained in 28.53 seconds
3_Decis

AutoML(mode='Compete')

In [10]:
model1_target['predict']=model1.predict(model1_target)

In [11]:
model2 = AutoML(mode='Compete')
model2.fit(model2_train_X, model2_train_y)

Linear algorithm was disabled.
AutoML directory: AutoML_8
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree logloss 0.147806 trained in 2.2 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree logloss 0.149258 trained in 16.6 seconds
2_DecisionTree logloss 0.145451 trained in 16.47 seconds
3_Decisio

AutoML(mode='Compete')

In [12]:
model2_target['predict']=model2.predict(model2_target)

In [13]:
pd.concat([model1_target, model2_target]).to_csv('./result/predict_result2.csv')